### Commands

In [106]:
from firecloud import api as firecloud_api
import glob
import fc_interface

In [107]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [118]:
### Parameters
namespace = "nci-mimoun-bi-org"
workspace = "CCLF_TSCA"
paths_to_samples_info = "paths_to_batches_info.xlsx"
google_bucket_id = "fc-35446f22-ea37-483a-bd6c-5e9fc56851ff"
batches_info = pd.read_excel(paths_to_samples_info)

#### Deletions

In [ ]:
### Delete unnecessary attributes in samples
attrs = ["annotate_variants_vcf", "annotate_variants_vcf_index", "oncotated_maf", "mutect2_vcf_index", "match_normal_oncotated_maf", "gnomad_vcf", "merged_vcfs", "match_primary_tumor_bam_file"]
for idx, row in all_samples.iterrows():
    fc_interface.delete_entity_attributes(namespace, workspace, "sample", row['entity:sample_id'], attrs)

In [ ]:
### Delete method repostory
firecloud_api.delete_repository_method('tsca', 'CreatePoN', 15)

In [ ]:
### Delete method repository
firecloud_api.delete_repository_method('tsca', 'Mutect2_for_Normal', 1)

In [ ]:
### Delete multiple method repositories
for i in range(10, 23):
    firecloud_api.delete_repository_method('tsca', 'CreatePoN', i)

In [ ]:
### Delete method config
firecloud_api.delete_workspace_config(namespace, workspace, "tsca", "Mutect2_for_Normal")

#### Pair creation

In [ ]:
### Create pairs
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

In [ ]:
### Upload pairs
res = fc_interface.upload_pairs(namespace, workspace, clean_pairs_list)

In [ ]:
# ### Delete unnecessary attributes in pairs
# ### NOT WOKING
# all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
# pairs_list = fc_interface.create_pairs_list(all_samples)
# attrs = ["__annotate_variants_vcf", "__annotate_variants_vcf_index", \
#          "__scattered_intervals", "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2", \
#          "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2"]
# for idx, row in pairs_list.iterrows():
#     fc_interface.delete_entity_attributes(namespace, workspace, "pair", row['entity:pair_id'], attrs)

In [ ]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

In [ ]:
### Delete sample_set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### All paths to batch info
# pd.read_excel('paths_to_batches_info.xlsx').path_to_samples_info.tolist()

In [ ]:
### Create cumulative PoN (all batches)
batches_info = pd.read_excel(paths_to_samples_info)
PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'CumPoN21', namespace, workspace)

In [ ]:
### Create PoN from normals of only one batch (no samples from other batches)
batches_info = pd.read_excel(paths_to_samples_info)
tsca21_path = batches_info.path_to_samples_info.tolist()[-1]
PoN = fc_interface.create_panel_of_normals([tsca21_path], -1, 'PoN_TSCA21_Normals')
fc_interface.upload_pon(PoN, 'PoN_TSCA21_Normals', namespace, workspace)

In [ ]:
### Delete pair set
fc_interface.delete_pair_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Delete sample set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Download method configs
fc_interface.download_method_configs(namespace, workspace)

In [ ]:
### Upload pairsets
tn_pairsets, tp_pairsets = fc_interface.create_pairsets(all_samples, clean_pairs_list)
fc_interface.upload_pairsets(namespace, workspace, tn_pairsets, "TN")
fc_interface.upload_pairsets(namespace, workspace, tp_pairsets, "TP")

In [ ]:
### Delete all pairs
for idx, row in clean_pairs_list.iterrows():
    res = fc_interface.delete_pair(namespace, workspace, row['entity:pair_id'])
    print(res)

In [ ]:
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

### Update pair attributes
update_pair_attrs(namespace, workspace, clean_pairs_list, attrs=['control_sample_tsca_id'])

In [ ]:
### Create PoN from all normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [6]:
### Create PoN with normals from batch + other normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
# PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
# fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [4]:
# batch_id = 'TSCA21'
# batch_path = batches_info.loc[batches_info.tsca_id==batch_id, 'path_to_samples_info'].item()
# pd.read_table(batch_path)
# batch_id, paths, N, name

In [21]:
# Retrieve samples before export to FC
tsca21 = \
    fc_interface.\
        prepare_batch_samples_for_metadata_export('/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                            'TSCA21', google_bucket_id)

In [ ]:
### Update WDLs
fc_interface.download_remote_wdls(namespace, workspace)

In [336]:
### Prepare cohorts for export to FC
cohorts = fc_interface.prepare_cohorts_for_metadata_export(paths_to_batches_info, google_bucket_id)

In [358]:
# # Delete cohorts
# cohort_codes = pd.read_table('cohort_files/cohort_names_dictionary.txt', header=None)
# for coh_code in cohort_codes.loc[:, 1].tolist():
#     res = fc_interface.delete_sample_set(namespace, workspace, coh_code)

In [338]:
# ### Upload metadata for given batch
# fc_interface.update_batch_metadata('TSCA21', \
#                                   '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
#                                       paths_to_samples_info, \
#                                         namespace, workspace, google_bucket_id)

65 Participants in this batch
95 Samples in this batch


In [361]:
# Export metadata
(r1, r2, r3, r4, r5, r6) = fc_interface.export_batch_metadata_to_fc('TSCA21', namespace, workspace)

In [359]:
# Cohorts
cohorts = fc_interface.prepare_cohorts_for_metadata_export(paths_to_batches_info, google_bucket_id)

In [66]:
### Create panel of normals from batch
pon = fc_interface.create_panel_of_normals_from_batch(batch_id, paths_to_samples_info, N=20).shape

In [78]:
### Create panel of normals from batch
for i, row in batches_info.iterrows():
    pon, name = fc_interface.create_panel_of_normals_from_batch(row['tsca_id'], paths_to_samples_info, N=20)
    fc_interface.upload_pon(pon, name, namespace, workspace)

In [ ]:
# Retrieve samples before export to FC
tsca21 = \
    fc_interface.\
        prepare_batch_samples_for_metadata_export('/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                            'TSCA21', google_bucket_id)

In [33]:
### Prepare cohorts for export to FC
cohorts = fc_interface.\
            prepare_cohorts_for_metadata_export(paths_to_samples_info, google_bucket_id, \
                                               blacklist=["CCLF_AA1012-Tumor-SM-F67DF"])

cohorts_sample_set_metadata = "cohort_files/fc_upload_sample_set_cohorts.txt"
res = fc_interface.upload_entities_from_tsv(namespace, workspace, cohorts_sample_set_metadata)

In [8]:
# Export metadata
(r1, r2, r3, r4, r5, r6) = fc_interface.export_batch_metadata_to_fc('TSCA21', namespace, workspace)

In [34]:
### Cohort of all tumors
res = fc_interface.upload_cohort_all_tumors(paths_to_samples_info, google_bucket_id, \
                                      'Cum_Tumors_22_all', namespace, workspace, ['DW039-Tumor-SM-DB2IF'])

In [12]:
# fc_interface.merge_walkupseq_files(paths_to_samples_info)

(1045, 14)

---

---

## Refined: New Batch Workflow

In [130]:
tsca_id = "TSCA22"
blacklist = ["DW039-Tumor-SM-DB2IF"]
path_to_batch_samples_info = batches_info.loc[batches_info.tsca_id==tsca_id, 'path_to_samples_info'].item()

In [29]:
### ALL SAMPLES
## Prepare
# fc_interface.prepare_patients_for_metadata_export(path_to_batch_samples_info, tsca_id)
# fc_interface.prepare_batch_sample_set_for_metadata_export(path_to_batch_samples_info, tsca_id)
# fc_interface.prepare_batch_samples_for_metadata_export(path_to_batch_samples_info, tsca_id, google_bucket_id)

## Export
# fc_interface.export_batch_metadata_to_fc(tsca_id, namespace, workspace)

In [39]:
### PANEL OF NORMALS BATCH
## Prepare 
# pon, name = fc_interface.create_panel_of_normals_from_batch(tsca_id, paths_to_samples_info, N=20)

# Export
# fc_interface.upload_pon(pon, name, namespace, workspace)

In [13]:
### PANEL OF NORMALS CUMULATIVE
## Prepare
# batches_info = pd.read_excel(paths_to_samples_info)
# PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'Cum_PoN_22_all')

# Export
# fc_interface.upload_pon(PoN, 'Cum_PoN_22_all', namespace, workspace)

Creating panel of 163 normals


<Response [200]>

In [132]:
# fc_interface.delete_sample_set(namespace, workspace, 'COHORT_AA')

In [133]:
### COHORT SAMPLE SETS
## Prepare
cohorts = fc_interface.prepare_cohorts_for_metadata_export(paths_to_samples_info, google_bucket_id, blacklist, tsca_id)

## Export
# res = fc_interface.upload_cohorts(namespace, workspace)

In [152]:
cohorts.shape

(1140, 20)

In [157]:
fc_interface.download_remote_samples(namespace, workspace)

In [158]:
remote_samples = pd.read_table('remote_samples.txt')

In [159]:
remote_samples.shape

(1140, 53)

In [161]:
remote_samples.head()

,entity:sample_id,aggregation_product_name_validation,bai_filename,bam_filename,bsp_sample_id_validation,chromosome_coverage_dist,clean_bai_file_capture,clean_bam_file_capture,cnv_calls,external_id_validation,...,oncotated_maf_mutect1,oncotated_maf_mutect2,segmented_copy_ratio_img,interval_list_padded,sample_cum_cov,sample_cum_cov_prop,clear_indels,filtered_indels,match_normal_bam_file,match_normal_sample_id
0,AA24-Tumor-SM-DHZ8W,TSCA Rapid Cancer Detection Panel v2,2_AA24T_OPAC_p9_H5V3YBCXY.2.aligned.duplicates...,2_AA24T_OPAC_p9_H5V3YBCXY.2.aligned.duplicates...,SM-DHZ8W,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/5...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/5...,AA24T_OPAC_p9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AA25-Tumor-SM-DHZ99,TSCA Rapid Cancer Detection Panel v2,2_AA25T_OPAC_p4_H5V3YBCXY.2.aligned.duplicates...,2_AA25T_OPAC_p4_H5V3YBCXY.2.aligned.duplicates...,SM-DHZ99,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/5...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/5...,AA25T_OPAC_p4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AA33-Normal-SM-D4L4D,TSCA Rapid Cancer Detection Panel v2,1_AA33N_3D_p5_HVLVWBCXX.1.aligned.duplicates_m...,1_AA33N_3D_p5_HVLVWBCXX.1.aligned.duplicates_m...,SM-D59KF,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/5...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/b...,AA33N_3D_p5,...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/f...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/f...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AA33-Normal-SM-DHZ9L,TSCA Rapid Cancer Detection Panel v2,2_AA33N_OPAC_p5_H5V3YBCXY.2.aligned.duplicates...,2_AA33N_OPAC_p5_H5V3YBCXY.2.aligned.duplicates...,SM-DHZ9L,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/5...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/5...,AA33N_OPAC_p5,...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/f...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/f...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AA33-Tumor-SM-DHZ9X,TSCA Rapid Cancer Detection Panel v2,2_AA33T_OPAC_p5_H5V3YBCXY.2.aligned.duplicates...,2_AA33T_OPAC_p5_H5V3YBCXY.2.aligned.duplicates...,SM-DHZ9X,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/5...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/5...,AA33T_OPAC_p5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [160]:
### PAIRS
cohorts.head()

,sample_id,participant_id,clean_bam_file_capture,external_id_validation,aggregation_product_name_validation,bsp_sample_id_validation,stock_sample_id_validation,sample_type,picard_aggregation_type_validation,processed_subtype_validation,source_subtype_validation,squid_sample_id_validation,tumor_subtype,short_letter_code,bam_filename,bai_filename,clean_bai_file_capture,tsca_id,Collection,membership:sample_set_id
0,AA33-Normal-SM-D4L4D,AA33,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA33N_3D_p5,TSCA Rapid Cancer Detection Panel v2,SM-D59KF,SM-D4L4D,Normal,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA33N_3D_p5,NaN,NDNA,1_AA33N_3D_p5_HVLVWBCXX.1.aligned.duplicates_m...,1_AA33N_3D_p5_HVLVWBCXX.1.aligned.duplicates_m...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA1213,Cancer Cell Line Factory (CCLF) / Andy Aguirre...,COHORT_AA
1,AA34-Tumor-SM-D4L25,AA34,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA34T_3D_p4,TSCA Rapid Cancer Detection Panel v2,SM-D59I2,SM-D4L25,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA34T_3D_p4,Metastatic,TM,1_AA34T_3D_p4_HVLVWBCXX.1.aligned.duplicates_m...,1_AA34T_3D_p4_HVLVWBCXX.1.aligned.duplicates_m...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA1213,Cancer Cell Line Factory (CCLF) / Andy Aguirre...,COHORT_AA
2,AA36-Tumor-SM-D4L2H,AA36,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA36T_3D_p5,TSCA Rapid Cancer Detection Panel v2,SM-D59IE,SM-D4L2H,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA36T_3D_p5,Metastatic,TM,1_AA36T_3D_p5_HVLVWBCXX.1.aligned.duplicates_m...,1_AA36T_3D_p5_HVLVWBCXX.1.aligned.duplicates_m...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA1213,Cancer Cell Line Factory (CCLF) / Andy Aguirre...,COHORT_AA
3,AA39-Tumor-SM-D4L3Q,AA39,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA39T_OPAC_3D_p5,TSCA Rapid Cancer Detection Panel v2,SM-D59JQ,SM-D4L3Q,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA39T_OPAC_3D_p5,Primary,TP,1_AA39T_OPAC_3D_p5_HVLVWBCXX.1.aligned.duplica...,1_AA39T_OPAC_3D_p5_HVLVWBCXX.1.aligned.duplica...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA1213,Cancer Cell Line Factory (CCLF) / Andy Aguirre...,COHORT_AA
4,AA40-Tumor-SM-D4L42,AA40,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA40T_3D_p5,TSCA Rapid Cancer Detection Panel v2,SM-D59K3,SM-D4L42,Tumor,PCR,Cells:Pellet frozen,Tissue:Fresh Tissue,AA40T_3D_p5,Primary,TP,1_AA40T_3D_p5_HVLVWBCXX.1.aligned.duplicates_m...,1_AA40T_3D_p5_HVLVWBCXX.1.aligned.duplicates_m...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA1213,Cancer Cell Line Factory (CCLF) / Andy Aguirre...,COHORT_AA
